In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xmitgcm 
from IPython.display import display, clear_output
import time

import os
import xarray as xr
import floater

import csv
import pandas as pd
from floater.generators import FloatSet

from matplotlib.colors import LogNorm


In [4]:
from dask.diagnostics import ProgressBar

In [5]:
def vector_gradient(x, y, x0, y0):
    ny, nx = x.shape
    grad = np.empty((ny,nx,2,2))
    

    x_p = np.roll(x, -1, axis=1)
    x_m = np.roll(x, 1, axis=1)
    x0_p = np.roll(x0, -1, axis=1)
    x0_m = np.roll(x0, 1, axis=1)

    y_p = np.roll(y, -1, axis=0)
    y_m = np.roll(y, 1, axis=0)
    y0_p = np.roll(y0, -1, axis=0)
    y0_m = np.roll(y0, 1, axis=0)

    dx = x_p - x_m
    dx0 = x0_p - x0_m

    dy = y_p - y_m
    dy0 = y0_p - y0_m
    
    
    grad[:,:,0,0] = dx / dx0
    grad[:,:,0,1] = dx / dy0
    grad[:,:,1,0] = dy / dx0
    grad[:,:,1,1] = dy / dy0
    return grad

def calc_l1_l2(x,y,x0,y0):
    GF = vector_gradient(x,y,x0,y0)
    C = np.empty_like(GF)

    C[:,:,0,0] = GF[:,:,0,0]**2 + GF[:,:,1,0]**2
    C[:,:,0,1] = GF[:,:,0,0]*GF[:,:,0,1] + GF[:,:,1,0]*GF[:,:,1,1]
    C[:,:,1,0] = GF[:,:,0,0]*GF[:,:,0,1] + GF[:,:,1,0]*GF[:,:,1,1]
    C[:,:,1,1] = GF[:,:,0,1]**2 + GF[:,:,1,1]**2

    # calculate trace and determinant
    Tr = C[:,:,0,0] + C[:,:,1,1]
    Det = C[:,:,0,0]*C[:,:,1,1] - C[:,:,0,1]*C[:,:,1,0]

    T = np.ma.masked_invalid(Tr)
    D = np.ma.masked_invalid(Det)

    rt = 0.25*T**2 - D
    if np.any(rt < 0):
        print('There will be imaginary eigenvalues')

    # large eigenvalue and eigenvector
    lam2 = 0.5*T + np.sqrt(rt)
    # these should all be greater than 1, mask otherwise
    mask = lam2<=1.

    # small eigenvalue and eigenvector
    # (direct computation is numerically unstable)
    lam1b = 0.5*T - np.sqrt(rt)

    # instead, compute as inverse of lam2
    lam1 = lam2**-1

    # apply mask to both
    lam1 = np.ma.masked_array(lam1, mask)
    lam2 = np.ma.masked_array(lam2, mask)
    return lam1, lam2

In [6]:
def calc_FTLE(ds,T_day,dT):
    #dscoords = {k: ds.coords[k] for k in ds.coords if k not in 'time'}
    #dsdims = {k: ds.dims[k] for k in ds.dims if k not in 'time'}
    #dscoords = ds.drop('time').coords 
    #dsdims = ds.drop('time').dims 
    
    Tstep = T_day * 24./dT
    l1,l2 = calc_l1_l2(ds.isel(time=int(Tstep)).x,ds.isel(time=int(Tstep)).y,ds.isel(time=0).x,ds.isel(time=0).y)
    
    FTLE = T_day**-1*np.log(l2)


    ftle = xr.DataArray(FTLE, coords = ds.drop('time').coords , dims=['y0','x0'])
    
    return ftle
    
def calc_LAVD(ds,T_day,dT_sample, dT_r,time_normal=False):
    end_slice = slice(int(dT_r*24/dT_sample-1),int(T_day*24/dT_sample), int(dT_r*24/dT_sample) )

    with ProgressBar():
        LAVD = (ds.lavd[end_slice].sum(dim='time')/(T_day*86400)).load()
    #LAVD = (ds.lavd[end_slice].sum(dim='time')/(T_day*86400))
    return LAVD

def calc_depth(ds, T_day, dT):
    depth = ds.isel(time=int(T_day*24./dT)).z.drop('time')
    return depth

In [7]:
codelist = ['100_3Dh','100_2Dh','100_3Dd','100_2Dd','100_3Dw','100_2Dw']

In [7]:
for i in range(6):

    code = codelist[i]

    indir = '/rigel/ocp/users/as4479/float_traj/%s/float_trajectories_netcdf/'%code


    %time ds = xr.open_mfdataset(indir+'*.nc')
    ds_all = ds.isel(time=slice(0,132))

    %time ds_diff = ds_all.diff('time')
    thresh = 10       # some large number of degrees, greater than any particle could
    # reasonably travel from one snapshot to the next
        
    %time mask = ((abs( ds_diff.y )>thresh) | (abs(ds_diff.x) > thresh)).sum(dim='time').load()

    ds_masked1 = ds_all.where(mask==0)
    ds_masked2 = ds_masked1.where(ds_masked1.vort!=0)
    
    durations = [15, 30, 60]
    FTLE = xr.concat([calc_FTLE(ds_masked2,dur,dT=12.) for dur in durations], 
                    dim=xr.Variable('duration', durations)).to_dataset(name='ftle')
    LAVD = xr.concat([calc_LAVD(ds_masked2,dur,dT_sample=12, dT_r=3) for dur in durations], 
                    dim=xr.Variable('duration', durations)).to_dataset(name='lavd')
    Depth = xr.concat([calc_depth(ds_masked2, dur, dT=12) for dur in durations], 
                    dim=xr.Variable('duration', durations)).to_dataset(name='depth')

    ds_lagr = xr.merge([FTLE, LAVD, Depth])
    ds_lagr.to_netcdf('/rigel/ocp/users/as4479/float_traj/ftle_lavd_depth_'+'%s.nc' %code, format='NETCDF4')
    
    print("Completed  %s" %code)
    print('------------')
    
    
    ds_lagr.close()
    #ds_masked1.close()
    #ds_masked2.close()
    #mask.close()
    ds.close()
    #ds_diff.close()
    

CPU times: user 1.73 s, sys: 166 ms, total: 1.9 s
Wall time: 6.06 s
CPU times: user 40 ms, sys: 253 µs, total: 40.2 ms
Wall time: 39.9 ms
CPU times: user 24 s, sys: 28.4 s, total: 52.4 s
Wall time: 47.9 s
Completed  100_3Dh
------------
CPU times: user 2.45 s, sys: 283 ms, total: 2.73 s
Wall time: 19.8 s
CPU times: user 40.8 ms, sys: 54 µs, total: 40.8 ms
Wall time: 40.9 ms
CPU times: user 24.5 s, sys: 28.8 s, total: 53.3 s
Wall time: 3min 26s


/rigel/ocp/users/as4479/new_miniconda/miniconda/envs/pangeo/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


Completed  100_2Dh
------------
CPU times: user 2.33 s, sys: 225 ms, total: 2.56 s
Wall time: 19.7 s
CPU times: user 114 ms, sys: 6.9 ms, total: 121 ms
Wall time: 121 ms
CPU times: user 23.9 s, sys: 29.3 s, total: 53.2 s
Wall time: 3min 11s
Completed  100_3Dd
------------
CPU times: user 2.36 s, sys: 203 ms, total: 2.57 s
Wall time: 20 s
CPU times: user 39 ms, sys: 293 µs, total: 39.3 ms
Wall time: 39.1 ms
CPU times: user 24.1 s, sys: 29.2 s, total: 53.3 s
Wall time: 3min 4s


/rigel/ocp/users/as4479/new_miniconda/miniconda/envs/pangeo/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


Completed  100_2Dd
------------
CPU times: user 2.25 s, sys: 259 ms, total: 2.51 s
Wall time: 21.3 s
CPU times: user 39.8 ms, sys: 105 µs, total: 39.9 ms
Wall time: 39.7 ms
CPU times: user 24 s, sys: 28.5 s, total: 52.6 s
Wall time: 3min


/rigel/ocp/users/as4479/new_miniconda/miniconda/envs/pangeo/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


Completed  100_3Dw
------------
CPU times: user 2.27 s, sys: 228 ms, total: 2.5 s
Wall time: 20.5 s
CPU times: user 38.4 ms, sys: 1.19 ms, total: 39.6 ms
Wall time: 39.7 ms
CPU times: user 24.1 s, sys: 29 s, total: 53 s
Wall time: 3min 2s


/rigel/ocp/users/as4479/new_miniconda/miniconda/envs/pangeo/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.


Completed  100_2Dw
------------


In [9]:
for i in range(6):

    code = codelist[i]

    indir = '/rigel/ocp/users/as4479/float_traj/%s/float_trajectories_netcdf/'%code


    %time ds = xr.open_mfdataset(indir+'*.nc')
    ds_all = ds.isel(time=slice(0,132))

    %time ds_diff = ds_all.diff('time')
    thresh = 10       # some large number of degrees, greater than any particle could
    # reasonably travel from one snapshot to the next
        
    %time mask = ((abs( ds_diff.y )>thresh) | (abs(ds_diff.x) > thresh)).sum(dim='time').load()

    ds_masked1 = ds_all.where(mask==0)
    ds_masked2 = ds_masked1.where(ds_masked1.vort!=0)
    
    durations = [15, 30, 60]
    
    LAVD = xr.concat([calc_LAVD(ds_masked2,dur,dT_sample=12, dT_r=3) for dur in durations], 
                    dim=xr.Variable('duration', durations)).to_dataset(name='lavd')

    LAVD.to_netcdf('/rigel/ocp/users/as4479/float_traj/lavd_'+'%s.nc' %code, format='NETCDF4')
    
    print("Completed  %s" %code)
    print('------------')
    
    

CPU times: user 1.71 s, sys: 143 ms, total: 1.85 s
Wall time: 6.78 s
CPU times: user 33.5 ms, sys: 0 ns, total: 33.5 ms
Wall time: 33.4 ms
CPU times: user 23.9 s, sys: 27.9 s, total: 51.8 s
Wall time: 26.2 s
Completed  100_3Dh
------------
CPU times: user 2.58 s, sys: 738 ms, total: 3.32 s
Wall time: 26.8 s
CPU times: user 39.1 ms, sys: 788 µs, total: 39.8 ms
Wall time: 39.8 ms
CPU times: user 23.9 s, sys: 29.5 s, total: 53.5 s
Wall time: 3min 42s
Completed  100_2Dh
------------
CPU times: user 2.38 s, sys: 260 ms, total: 2.64 s
Wall time: 23.4 s
CPU times: user 40.4 ms, sys: 0 ns, total: 40.4 ms
Wall time: 40.4 ms
CPU times: user 23.5 s, sys: 28.9 s, total: 52.3 s
Wall time: 3min 7s
Completed  100_3Dd
------------
CPU times: user 2.46 s, sys: 222 ms, total: 2.68 s
Wall time: 19.5 s
CPU times: user 39.8 ms, sys: 118 µs, total: 39.9 ms
Wall time: 39.6 ms
CPU times: user 23.7 s, sys: 28.9 s, total: 52.6 s
Wall time: 3min 1s
Completed  100_2Dd
------------
CPU times: user 2.28 s, sys: 207

In [11]:
code = codelist[5]

indir = '/rigel/ocp/users/as4479/float_traj/%s/float_trajectories_netcdf/'%code


%time ds = xr.open_mfdataset(indir+'*.nc')
ds_all = ds.isel(time=slice(0,132))

%time ds_diff = ds_all.diff('time')
thresh = 10       # some large number of degrees, greater than any particle could
# reasonably travel from one snapshot to the next
        
%time mask = ((abs( ds_diff.y )>thresh) | (abs(ds_diff.x) > thresh)).sum(dim='time').load()

ds_masked1 = ds_all.where(mask==0)
ds_masked2 = ds_masked1.where(ds_masked1.vort!=0)
    
durations = [15, 30, 60]
    
LAVD = xr.concat([calc_LAVD(ds_masked2,dur,dT_sample=12, dT_r=3) for dur in durations], 
                dim=xr.Variable('duration', durations)).to_dataset(name='lavd')

LAVD.to_netcdf('/rigel/ocp/users/as4479/float_traj/lavd_'+'%s.nc' %code, format='NETCDF4')
    
print("Completed  %s" %code)
print('------------')
    

CPU times: user 1.51 s, sys: 127 ms, total: 1.64 s
Wall time: 4.7 s
CPU times: user 163 ms, sys: 14.3 ms, total: 178 ms
Wall time: 182 ms
CPU times: user 24 s, sys: 28.2 s, total: 52.2 s
Wall time: 51.7 s
[########################################] | 100% Completed |  1.2s
[########################################] | 100% Completed |  2.3s
[########################################] | 100% Completed |  7.4s
Completed  100_2Dw
------------


In [10]:
# Calculate the 1, 5, 10 Day LAVD fields

In [9]:
def calc_LAVD(ds,T_day,dT_sample, dT_r,time_normal=False):
    if (T_day < dT_r):
        end_slice = slice(0,int(T_day*24/dT_sample))
        LAVD = (ds.lavd.isel(time=int(T_day*24/dT_sample)).drop('time')/(T_day*86400)).load()
    elif (T_day < 2* dT_r):
        end_slice = slice(int(dT_r*24/dT_sample-1),int(T_day*24/dT_sample), 2)
        LAVD = (ds.lavd[end_slice].sum(dim='time')/(T_day*86400)).load()
    else:
        end_slice = slice(int(dT_r*24/dT_sample-1),int(T_day*24/dT_sample), int(dT_r*24/dT_sample) )
        with ProgressBar():
            LAVD = ((ds.lavd[end_slice].sum(dim='time') + ds.lavd.isel(time=int(T_day*24/dT_sample)).drop('time'))/(T_day*86400)).load()
    #LAVD = (ds.lavd[end_slice].sum(dim='time')/(T_day*86400))
    return LAVD

In [11]:
for i in range(6):

    code = codelist[i]

    indir = '/rigel/ocp/users/as4479/float_traj/%s/float_trajectories_netcdf/'%code


    %time ds = xr.open_mfdataset(indir+'*.nc')
    ds_all = ds.isel(time=slice(0,132))

    %time ds_diff = ds_all.diff('time')
    thresh = 10       # some large number of degrees, greater than any particle could
    # reasonably travel from one snapshot to the next
        
    %time mask = ((abs( ds_diff.y )>thresh) | (abs(ds_diff.x) > thresh)).sum(dim='time').load()

    ds_masked1 = ds_all.where(mask==0)
    ds_masked2 = ds_masked1.where(ds_masked1.vort!=0)
    
    durations = [1, 5, 10]
    
    LAVD = xr.concat([calc_LAVD(ds_masked2,dur,dT_sample=12, dT_r=3) for dur in durations], 
                    dim=xr.Variable('duration', durations)).to_dataset(name='lavd')

    LAVD.to_netcdf('/rigel/ocp/users/as4479/float_traj/lavd_1_5_10_'+'%s.nc' %code, format='NETCDF4')
    
    print("Completed  %s" %code)
    print('------------')

CPU times: user 2.29 s, sys: 503 ms, total: 2.79 s
Wall time: 28.1 s
CPU times: user 37.8 ms, sys: 1.78 ms, total: 39.6 ms
Wall time: 39.7 ms
CPU times: user 23.3 s, sys: 29.2 s, total: 52.5 s
Wall time: 3min 28s
[########################################] | 100% Completed |  0.9s
Completed  100_3Dh
------------
CPU times: user 2.55 s, sys: 457 ms, total: 3.01 s
Wall time: 29.3 s
CPU times: user 39.7 ms, sys: 0 ns, total: 39.7 ms
Wall time: 40 ms
CPU times: user 23.2 s, sys: 29.1 s, total: 52.3 s
Wall time: 4min 28s
[########################################] | 100% Completed |  1.0s
Completed  100_2Dh
------------
CPU times: user 2.51 s, sys: 437 ms, total: 2.95 s
Wall time: 34.9 s
CPU times: user 40.7 ms, sys: 0 ns, total: 40.7 ms
Wall time: 40.8 ms
CPU times: user 23.2 s, sys: 28.9 s, total: 52.1 s
Wall time: 3min 52s
[########################################] | 100% Completed |  0.9s
Completed  100_3Dd
------------
CPU times: user 2.39 s, sys: 422 ms, total: 2.81 s
Wall time: 29.1 s
